In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# Предполагаем, что у вас уже есть данные X и y
# Разделим данные на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Функция для оптимизации
def objective(trial):
    # Гиперпараметры, которые будем настраивать
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    
    # Создаем модель с текущими гиперпараметрами
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
        class_weight="balanced"  # Учитываем дисбаланс классов
    )
    
    # Обучаем модель
    rf.fit(X_train, y_train)
    
    # Предсказания на тестовой выборке
    y_pred = rf.predict(X_test)
    
    # Оцениваем метрику (например, F1-метрику)
    f1 = f1_score(y_test, y_pred, average="weighted")  # Учитываем дисбаланс
    
    return f1

# Запуск Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Best trial:")
print(study.best_trial)
print("Best params:")
print(study.best_params)

# Используем лучшие параметры для финальной модели
best_params = study.best_params
final_rf = RandomForestClassifier(**best_params, random_state=42, class_weight="balanced")
final_rf.fit(X_train, y_train)
